## Imports

In [50]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [51]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [52]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [53]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [54]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
1,Milano at Deering Bay\r13621 Deering Bay Dr PH...,12/16/2019,7/18/2022,812,"$2,650,000.00",4067,$651.59,Istvan Zolcsak,One Sotheby's INT'L Realty,MariaClaudia Fernandez,One Sotheby's INT'L Realty,Milano at Deering Bay 13621 Deering Bay Dr PH1...,Milano at Deering Bay,13621 Deering Bay Dr PH1202 Coral Gables
2,Acqualina Ocean Residence\r17875 Collins Ave 9...,5/11/2022,7/20/2022,33,"$2,400,750.00",2214,"$1,084.35",Carole Perez,"Finvarb Realty, Inc.",Sabina Furman,AG Lux Group,Acqualina Ocean Residence 17875 Collins Ave 90...,Acqualina Ocean Residence,17875 Collins Ave 905 Sunny Isles Beach
3,Villa Harbour Apts Condo\r166 Harbor Dr 7C\rKe...,6/16/2022,7/22/2022,3,"$2,100,000.00",1408,"$1,491.48",Shane Graber,Graber Realty Group LLC,Shane Graber,Graber Realty Group LLC,Villa Harbour Apts Condo 166 Harbor Dr 7C Key ...,Villa Harbour Apts Condo,166 Harbor Dr 7C Key Biscayne
4,One Paraiso\r3131 NE 7th Ave 4206\rMiami,4/4/2022,7/18/2022,49,"$1,885,000.00",1634,"$1,153.61",Adam Redolfi,BARNES INT'L REALTY LLC,Marko Gojanovic,One Sotheby's INT'L Realty,One Paraiso 3131 NE 7th Ave 4206 Miami,One Paraiso,3131 NE 7th Ave 4206 Miami
5,Murano Grande\r400 Alton Rd TH-7M\rMiami Beach,5/23/2022,7/17/2022,10,"$1,730,000.00",1310,"$1,320.61",Tracy Galya,Douglas Elliman,Kristie Weyhmiller,One Sotheby's INT'L Realty,Murano Grande 400 Alton Rd TH-7M Miami Beach,Murano Grande,400 Alton Rd TH-7M Miami Beach
6,SLS Lux\r801 S Miami Ave 2605\rMiami,3/1/2022,7/19/2022,87,"$1,600,000.00",2931,$658.98,Devin Kay,Douglas Elliman,Matt Lill,The Corcoran Group,SLS Lux 801 S Miami Ave 2605 Miami,SLS Lux,801 S Miami Ave 2605 Miami
7,Key Colony Oceansound\r251 Crandon Blvd 306\rK...,5/19/2022,7/18/2022,23,"$1,560,000.00",1546,"$1,009.06",Martha Ledesma Guevara,EXP Realty LLC,Giulietta Ulloa,BHHS EWM Realty,Key Colony Oceansound 251 Crandon Blvd 306 Key...,Key Colony Oceansound,251 Crandon Blvd 306 Key Biscayne
8,Trump Tower III\r15811 Collins Ave 3803\rSunny...,6/14/2022,7/20/2022,6,"$1,550,000.00",1935,$801.03,Anastasia Philippova,London Foster Realty,David Veit PA,Brown Harris Stevens,Trump Tower III 15811 Collins Ave 3803 Sunny I...,Trump Tower III,15811 Collins Ave 3803 Sunny Isles Beach
9,Bel-Air on the Ocean\r6315 Collins Ave 1203\rM...,5/27/2022,7/18/2022,7,"$1,295,000.00",1126,"$1,150.09",Glen Janney,Cervera Real Estate Inc.,Michele Redlich,Coldwell Banker Realty,Bel-Air on the Ocean 6315 Collins Ave 1203 Mia...,Bel-Air on the Ocean,6315 Collins Ave 1203 Miami Beach
10,O0 ceana IV Condo\r16400 Collins Ave 2642\rSun...,6/3/2022,7/20/2022,12,"$1,175,000.00",1530,$767.97,Boris Teplitsky,London Foster Realty,Hernan Vergara,Fortune Int'l Realty Aventura,O0 ceana IV Condo 16400 Collins Ave 2642 Sunny...,O,0 ceana IV Condo 16400 Collins Ave 2642 Sunny ...


In [55]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [56]:
### Needed to add the +1 because the index always starts at 1 instead of
### zero.
df.loc[len(df.index) + 1] = ['asia_condo','9/9/2020','7/21/2022','673','$7,000,000','square_feet','$1,413','Jordan Lederman','Fortune International Realty','Jorge Rico & Andrea Viana','Compass Florida','geo_address','Asia condo','900 Brickell Key Boulevard, PH3401']

## Geocoder Setup and Execution

In [57]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [58]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [59]:
df['loc']

1     (13621 Deering Bay Dr PH 1202, Coral Gables, F...
2     (17875 Collins Ave #905, Sunny Isles Beach, FL...
3     (166 Harbor Dr #7c, Key Biscayne, FL 33149, US...
4     (3131 NE 7th Ave #4206, Miami, FL 33137, USA, ...
5     (400 Alton Rd #7m, Miami Beach, FL 33139, USA,...
6     (801 S Miami Ave # 2605a, Miami, FL 33130, USA...
7     (251 Crandon Blvd APT 306, Key Biscayne, FL 33...
8     (15811 Collins Ave APT 3803, Sunny Isles Beach...
9     (6315 Collins Ave #1203, Miami Beach, FL 33141...
10    (16400 Collins Ave APT 2642, Sunny Isles Beach...
11    (900 Brickell Key Blvd, Miami, FL 33131, USA, ...
Name: loc, dtype: object

In [60]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [61]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Milano at Deering Bay\r13621 Deering Bay Dr PH...,12/16/2019,7/18/2022,812,"$2,650,000.00",4067,$651.59,Istvan Zolcsak,One Sotheby's INT'L Realty,MariaClaudia Fernandez,One Sotheby's INT'L Realty,Milano at Deering Bay 13621 Deering Bay Dr PH1...,Milano at Deering Bay,13621 Deering Bay Dr PH1202 Coral Gables,"(13621 Deering Bay Dr PH 1202, Coral Gables, F...","(25.6376854, -80.2909919, 0.0)",25.637685,-80.290992,0.0
2,Acqualina Ocean Residence\r17875 Collins Ave 9...,5/11/2022,7/20/2022,33,"$2,400,750.00",2214,"$1,084.35",Carole Perez,"Finvarb Realty, Inc.",Sabina Furman,AG Lux Group,Acqualina Ocean Residence 17875 Collins Ave 90...,Acqualina Ocean Residence,17875 Collins Ave 905 Sunny Isles Beach,"(17875 Collins Ave #905, Sunny Isles Beach, FL...","(25.940858, -80.1208766, 0.0)",25.940858,-80.120877,0.0
3,Villa Harbour Apts Condo\r166 Harbor Dr 7C\rKe...,6/16/2022,7/22/2022,3,"$2,100,000.00",1408,"$1,491.48",Shane Graber,Graber Realty Group LLC,Shane Graber,Graber Realty Group LLC,Villa Harbour Apts Condo 166 Harbor Dr 7C Key ...,Villa Harbour Apts Condo,166 Harbor Dr 7C Key Biscayne,"(166 Harbor Dr #7c, Key Biscayne, FL 33149, US...","(25.6997741, -80.16801939999999, 0.0)",25.699774,-80.168019,0.0
4,One Paraiso\r3131 NE 7th Ave 4206\rMiami,4/4/2022,7/18/2022,49,"$1,885,000.00",1634,"$1,153.61",Adam Redolfi,BARNES INT'L REALTY LLC,Marko Gojanovic,One Sotheby's INT'L Realty,One Paraiso 3131 NE 7th Ave 4206 Miami,One Paraiso,3131 NE 7th Ave 4206 Miami,"(3131 NE 7th Ave #4206, Miami, FL 33137, USA, ...","(25.8070469, -80.1855776, 0.0)",25.807047,-80.185578,0.0
5,Murano Grande\r400 Alton Rd TH-7M\rMiami Beach,5/23/2022,7/17/2022,10,"$1,730,000.00",1310,"$1,320.61",Tracy Galya,Douglas Elliman,Kristie Weyhmiller,One Sotheby's INT'L Realty,Murano Grande 400 Alton Rd TH-7M Miami Beach,Murano Grande,400 Alton Rd TH-7M Miami Beach,"(400 Alton Rd #7m, Miami Beach, FL 33139, USA,...","(25.7732967, -80.1403323, 0.0)",25.773297,-80.140332,0.0
6,SLS Lux\r801 S Miami Ave 2605\rMiami,3/1/2022,7/19/2022,87,"$1,600,000.00",2931,$658.98,Devin Kay,Douglas Elliman,Matt Lill,The Corcoran Group,SLS Lux 801 S Miami Ave 2605 Miami,SLS Lux,801 S Miami Ave 2605 Miami,"(801 S Miami Ave # 2605a, Miami, FL 33130, USA...","(25.7661392, -80.19290869999999, 0.0)",25.766139,-80.192909,0.0
7,Key Colony Oceansound\r251 Crandon Blvd 306\rK...,5/19/2022,7/18/2022,23,"$1,560,000.00",1546,"$1,009.06",Martha Ledesma Guevara,EXP Realty LLC,Giulietta Ulloa,BHHS EWM Realty,Key Colony Oceansound 251 Crandon Blvd 306 Key...,Key Colony Oceansound,251 Crandon Blvd 306 Key Biscayne,"(251 Crandon Blvd APT 306, Key Biscayne, FL 33...","(25.6977929, -80.1579812, 0.0)",25.697793,-80.157981,0.0
8,Trump Tower III\r15811 Collins Ave 3803\rSunny...,6/14/2022,7/20/2022,6,"$1,550,000.00",1935,$801.03,Anastasia Philippova,London Foster Realty,David Veit PA,Brown Harris Stevens,Trump Tower III 15811 Collins Ave 3803 Sunny I...,Trump Tower III,15811 Collins Ave 3803 Sunny Isles Beach,"(15811 Collins Ave APT 3803, Sunny Isles Beach...","(25.9232597, -80.1218135, 0.0)",25.923260,-80.121814,0.0
9,Bel-Air on the Ocean\r6315 Collins Ave 1203\rM...,5/27/2022,7/18/2022,7,"$1,295,000.00",1126,"$1,150.09",Glen Janney,Cervera Real Estate Inc.,Michele Redlich,Coldwell Banker Realty,Bel-Air on the Ocean 6315 Collins Ave 1203 Mia...,Bel-Air on the Ocean,6315 Collins Ave 1203 Miami Beach,"(6315 Collins Ave #1203, Miami Beach, FL 33141...","(25.8449695, -80.1204059, 0.0)",25.844970,-80.120406,0.0
10,O0 ceana IV Condo\r16400 Collins Ave 2642\rSun...,6/3/2022,7/20/2022,12,"$1,175,000.00",1530,$767.97,Boris Teplitsky,London Foster Realty,Hernan Vergara,Fortune Int'l Realty Aventura,O0 ceana IV Condo 16400 Collins Ave 2642 Sunny...,O,0 ceana IV Condo 16400 Collins Ave 2642 Sunny ...,"(16400 Collins Ave A

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [68]:
df.at[10,'building_name']=('Oceana IV Condo')
df.at[10,'address_only']=('16400 Collins Ave 2642')

In [69]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('July 17th - July 23rd')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [64]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [65]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [66]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesWeekEnding07_23_22
